# Vectorization and the Numpy Functional API

The goal of this section is to introduce you to some of the functions in the numpy library and to introduce the concept of vectorization, which allows you to write performant and clean code. To kick off this section, here is an example that motivates why you want to learn about vectorization. Below you can find three versions of a function that adds a number to each element in a 2d array (matrix). The first uses a traditional list syntax, the second does the list-style additions on a `ndarray`, and the third one uses vectorization and the numpy API. Each addition is executed 100 times

**Note**: This example takes about 30s to complete on my computer. You can run it, and study the code while it executes.

In [ ]:
from typing import List, Tuple
import numpy as np
import timeit


def add_to_matrix_list(matrix: List[List[float]], number: float, *, shape: Tuple[int, int]) -> List[List[float]]:
    # equivalent to np.zeros_like(matrix)
    result = [[0] * shape[1] for y in range(shape[0])]
    
    # equivalent of matrix + number
    for y in range(shape[0]):
        for x in range(shape[1]):
            result[y][x] = matrix[y][x] + number

    return result


def add_to_matrix(matrix: np.ndarray, number: float) -> np.ndarray:
    result = np.zeros_like(matrix)

    # equivalent of matrix + number
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            result[y, x] = matrix[y, x] + number

    return result


def add_to_matrix_numpy(matrix: np.ndarray, number: float) -> np.ndarray:
    return matrix + number


matrix_shape = (1000, 1000)
some_array = np.arange(1000*1000).reshape(1000, 1000)
some_list = some_array.tolist()

manual_as_list = add_to_matrix_list(some_list, 2, shape=(100,100))
manual_as_ndarray = add_to_matrix(some_array, 2)
via_numpy = add_to_matrix_numpy(some_array, 2)

time_list = timeit.timeit("add_to_matrix_list(some_list, 2, shape=(1000,1000))", "from __main__ import some_list, add_to_matrix_list", number=100)
time_array = timeit.timeit("add_to_matrix(some_array, 2)", "from __main__ import some_array, add_to_matrix", number=100)
time_numpy = timeit.timeit("add_to_matrix_numpy(some_array, 2)", "from __main__ import some_array, add_to_matrix_numpy", number=100)

print(f"""Timings for different runs:
List: {time_list/10:>16.6f} ms
Manual Array: {time_array/10:>7.6f} ms
Numpy: {time_numpy/10:>15.6f} ms

 """)


On my machine the list version takes about 0.5 ms to process a (1000, 1000) array. If we only need to do it a few times this is certainly something we can live with; on the other hand, if it happens along the critical path (the sequence of code that takes the most time in our current program) then we might wish to optimize it. This is where numpy shines, which - again on my machine - is a whooping 90x faster. Perhaps even more surprisingly, numpy's version of `array + number` looks very similar to the above loops if you peak under the hood.

So why is numpy so much faster? The short answer is *optimization*. The not so short and very technical answer is that there are multiple factors contributing to this. The first is that numpy calls a C function under the hood. C code tends to be faster than python code, because (a) C code can do compile-time optimizations, and (b) C code doesn't have to check for exceptions after each step. The next speed boost comes from a different memory layout. A python list is a sequence of objects that can - essentially - live in different locations of your RAM, whereas numpy arrays are guaranteed to occupy a single continous block of memory. Instead of checking where the next element is located we can directly go to the neighbouring memory block, which - again - makes our code much faster. Thirdly, because of this difference in memory layout we can use something called SIMD (sometimes called vectorization extension), a special ability of many modern CPUs to process a block of numbers in one step. Finally, being in C allows us to side-step the python GIL and use threads that actually run in parallel. While numpy itself doesn't do multi-threadding, some of the libraries that it builds on (MKL/OpenBLAS) might.

The second observation we can make is that manually adding to a `ndarray` is about 4x slower than the list-of-list version. This highlights a second important aspect of the numpy API. Numpy is implemented in C, and there is a constant overhead associated with calling a C function from python. This overhead is so small, that we don't have to worry about it if we do few calls. However, if we call a numpy function 1 million times (1000 * 1000) in a loop this overhead becomes painfully noticable.

## Vectorization

A typical description of vectorization is *to remove the for loops from your code*. However, a more accurate description of vectorization is *replace parts of your code with function calls from a highly-optimized library*. It is about letting your computer know that you want to perform a certain operation repeatedly on every element in some structure, and that this operation is pure in the sense that it will have no side-effects on other variables in our code. Most languages offer libraries tailored to this purpose. Within such a library you usually find an array of functions that correspond to primitive operations (plus, minus, multiply, divide, modulo, sin, cos, ...). Each function typically takes in one or more special objects, e.g. `ndarray`s, and performs the desired operation for you on every element in the structure and returns the result. From your (a users) perspective, however, it really is a game of *which piece of my program can I replace with a function call from this library* and - usually - every for loop that does numerical processing is a good place to start.

Numpy makes this job particularily easy, because it has a symbiotic and insanley deep relationship with the language itself. Some parts of the python language, e.g. Ellipsis (...), exist in python because the numpy community asked for it). As a result, most basic python operators (`+`, `-`, `%`, ...) are overloaded for numpy arrays and they will automatically call the optimized function for you without you having to worry about it explicitly. Most other functions are available in numpy's primary namespace, e.g., `np.sin`, `np.dot`, `np.sum`, and many more. These are called `ufunc`s which stands for universal functions.

Here is a list of all the overloaded operators



In [ ]:
# working with constants happens element wise
vector = np.ones(5)
a0 = vector + 4
a1 = vector - 1
a2 = vector * 4
a3 = vector / 4
a4 = vector // 4 # integer division
a5 = vector % 4
a6 = vector ** 7

# boolean operations work element-wise, too
bool_vector = np.array([1, 0, 1, 0], dtype=bool)
a7 = bool_vector & bool_vector
a8 = bool_vector | bool_vector
a9 = bool_vector ^ bool_vector
a10 =  vector.astype(int) >> 3
a11 = vector.astype(int) << 3

# logical operations work element-wise, too
vector2 = np.array([52, -3, 4, 7, -42])
a12 = vector == vector2
a13 = vector > vector2
a14 = vector < vector2
a15 = vector != vector2

# of course you can do this with any array
# and syntax-sugar like += is supported
tensor = np.arange(32*256*256*3).reshape(32, 256, 256, 3)
tensor += 42
tensor %= 42

# working with two arrays will apply the operation element-wise
matrix = np.arange(25, dtype=float).reshape(5, 5)
matrix2 = np.linspace(1, 100, 25).reshape(5, 5)

a16 = matrix ** matrix2
matrix -= matrix2

# finally, a special operator you may be unaware of
a17 = matrix @ vector  # matrix-vector multiplication


With this we can already do some basic vectorization (and de-vectorization for the sake of practice).

### Exercise 1

Vectorize the following code snippets

In [ ]:
# Exercise 1.1
# vectorize the snippet below
foo = np.ones((3, 6, 9, 12)).tolist()
for x1 in range(3):
    for x2 in range(6):
        for x3 in range(9):
            for x4 in range(12):
                element = foo[x1][x2][x3][x4]
                new_element = element + 5
                new_element /= 3
                foo[x1][x2][x3][x4] = new_element

<details>
<summary>Answer (click me to reveal)</summary>

```
foo = (foo + 5) / 3
```
</details>

In [ ]:
# Exercise 1.2
# vectorize the snippet below
foo = [1, 3, 3, 7, 15]
bar = [6.0, 5.0, 4.0, 3.0, 2.0]

baz = list()
for x, y in zip(foo, bar):
    baz.append(x - 10*y)

<details>
<summary>Answer (click me to reveal)</summary>

```
baz = foo - 10 * bar 
```
</details>

In [ ]:
# Exercise 1.3
# de-vectorize this expression, i.e. write it in plain python code
foo = np.arange(100) % 2 == 0

<details>
<summary>Answer (click me to reveal)</summary>

Using list comprehension

```
foo = [x % 2 == 0 for x in range(100)]
```

more manually

```
foo = list()
for x in range(100):
    foo.append(x % 2 == 0)
```
</details>

In [ ]:
# Exercise 1.4
# vectorize the snippet below
foo = list()
for y in range(12):
    sub_list = list()
    for x in range(10):
        if x > y:
            sub_list.append(0)
        else:
            sub_list.append(42)

    foo.append(sub_list)


<details>
<summary>Answer (click me to reveal)</summary>

```
42 * np.tri(12, 10, dtype=int)
```
</details>

In [ ]:
# Exercise 1.3
# de-vectorize this expression, i.e. write it in plain python code
foo = ((np.arange(100) % 2) * 2 - 1) * np.arange(100)

<details>
<summary>Answer (click me to reveal)</summary>

```
foo = list()
for idx in range(100):
    if idx % 2 == 0:
        foo.append(-idx)
    else:
        foo.append(idx)
```

with list comprehension

```
foo = [x if x % 2 else -x for x in range(100)]
```

</details>

## The Functional API

As hinted above, playing the game of *vectorization* essentially means replacing segments of your code with function calls to a highly-optimized library (like numpy). Above, we were mainly looking at replacing operations on individual elements. However, nothing stops us from extending this notion to operations on individual rows, individual columns, or more generally, individual axes of an array. For example, we might wish to sum up every row in a matrix. We may also wish to do this for groups of two or more arrays, e.g., we might wish to compute a matrix-vector multiplication (`@`) or we may wish to compute the dot product of two (or more vectors). We can even look beyond that and think about operations that happen along two axes, where we can find complex operations such as matrix decompositions or convolutions.

The constantly growing list of numpy routines can be found in the documentation: [https://numpy.org/devdocs/reference/routines.html](https://numpy.org/devdocs/reference/routines.html)


Here are some examples that you will encounter frequently.

In [ ]:
foo = np.linspace(1, 10, 20)
bar = np.arange(1, 21)

# some more element-wise functions
a0 = np.cos(foo)
a1 = np.floor(foo)
a2 = np.clip(bar, 5, 15)
a3 = np.exp(bar)
a4 = np.log(foo)
a5 = np.sqrt(foo)

# functions that work along an axis
a6 = np.sum(foo)        # the sum of all elements in the array
a7 = np.mean(foo)
a8 = np.std(foo)        # standard deviation
a9 = np.var(foo)        # variance
a10 = np.dot(foo, bar)  # scalar product
a11 = np.any(foo.astype(bool))  # True iff any element of foo is True
a12 = np.all(bar.astype(bool))  # True iff all elements of bar are True     


Functions that operate along an axis flatten the input by default, i.e., if you call `np.sum(np.eye(6))`, numpy will compute `np.sum(np.eye(6).ravel()) == 6`. If you want the operation to be computed along a specific axis and simply iterate over the remaining ones, you can specify the axis in question using the optional keyword argument `axis=`. For example `np.sum(np.eye(6), axis=1).tolist()` will sum over each row in the matrix and produce `[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]`.

### Exercise 2

1. Normalize `vectorA`.
2. Replace the largest value in the `vectorA` with 5
3. Extract the diagonal of the matrix product of `matrixA` and `matrixB`.
4. Write the following expression as a sequence of numpy functions: $\text{mse} = \sum_{i=0}^N (y - (W\cdot x + c))^2$

In [ ]:
vectorA = np.array([-2, 8, 3, 9])
matrixA = np.kron(np.array([1, 2, 3])[:, None], np.array([4, 5, 6])[None, :])
matrixB = np.arange(10, 1, -1).reshape(3, 3)

y = np.array([1, 2, 3])
W = matrixA
x = np.array([43, -1, 7])
c = np.array([1, 0, 1])

<details>
<summary>Answer (click me to reveal)</summary>

Exercise 2.1
```
vectorA = vectorA / np.sqrt(np.sum(vectorA ** 2))
```

Exercise 2.2
```
idx = np.argmax(vectorA)
vectorA[idx] = 5
```

Exercise 2.3
```
np.diag(matrixA @ matrixB)

# Faster version
np.sum(matrixA * matrixB.T, axis=1)

# Even Faster version
np.einsum("ij,ji->i", matrixA, matrixB)
```

Exercise 2.4
```
mse = np.sum((y - (W @ x + c)) ** 2)
```
</details>

## Final Exercises

### Exercise 3

The array `weather_data` contians daily Swedish weather data from Jan-2020 until approximately the end of Apr-2020. Each row in the data matrix corresponds to a day, and each column in the data matrix contains a different feature of the weather. In more detail, the four columns contain `[temperatureLow, temperatureHigh, humidity, windSpeed]` in that order. Unfortunately, the temperature is measured in °F (Farenheit) instead of °C (Celsius).

Compute the minimum and maximum temperature in Celsius (°C) for each day. Then compute the overall average temperature as well as the temperature's standard deviation.

In [ ]:
weather_data = np.load("weather_data.npy")

<details>
<summary>Answer (click me to reveal)</summary>

Exercise 3
```
weather_converted = weather_data.copy()
temp_in_C = (weather_data[:, :2] - 32 ) * 5/9
weather_converted[:, :2] = temp_in_C

mean_temperature = np.mean(temp_in_C)
standard_deviation = np.std(temp_in_C)
```
</details>